In [10]:
# Import the necessary modules
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table

from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
from crud_module import CRUD

# FIX ME: Update the MongoDB connection details
USER = 'root'
PASS = 'hbaG98WkC8'
HOST = 'nv-desktop-services.apporto.com'
PORT = '31902'
DB = 'aac'
COL = 'animals'

# Set up the CRUD instance
shelter = CRUD(USER, PASS, HOST, PORT, DB, COL)

# Read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the '_id' column
df.drop(columns=['_id'], inplace=True)

# Set up the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        style_table={'height': '400px', 'overflowY': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

# Define callback to highlight selected row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is not None:
        return [{
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    else:
        # Return an empty list if selected_columns is None
        return []


# Define callback to update the map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is not None and index is not None and len(index) > 0:
        selected_row = viewData[index[0]]
        
        # Check if 'Latitude' and 'Longitude' keys exist in the selected row
        if 'Latitude' in selected_row and 'Longitude' in selected_row:
            latitude = selected_row['Latitude']
            longitude = selected_row['Longitude']
            
            return [
                dl.Map(style={'width': '1000px', 'height': '500px'},
                       center=[latitude, longitude], zoom=10, children=[
                        dl.TileLayer(id="base-layer-id"),
                        dl.Marker(position=[latitude, longitude],
                                  children=[
                                      dl.Tooltip(selected_row.get('Breed', '')),
                                      dl.Popup([
                                          html.H1("Animal Name"),
                                          html.P(selected_row.get('Name', ''))
                                      ])
                                  ])
                    ])
            ]
    
    # Return an empty map if no row is selected or if 'Latitude' or 'Longitude' keys are missing
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10,
                   children=[dl.TileLayer(id="base-layer-id")])]

# Run the server
app.run_server(debug=True)
